Автор материала: Юрий Кашницкий (@yorko в Slack ODS). Материал распространяется на условиях лицензии Creative Commons CC BY-NC-SA 4.0.

# Домашнее задание 6 (демо). Линейная регрессия, Lasso и RF-регрессия в задаче по определению качества вина

In [15]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
from sklearn.metrics.regression import mean_squared_error
from sklearn.model_selection import (GridSearchCV, cross_val_score,
                                     train_test_split)
from sklearn.preprocessing import StandardScaler

Будем работать с набором данных по качеству белого вина (репозиторий UCI). Загружаем данные.

In [16]:
data = pd.read_csv('wine_quality.csv')

In [17]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
fixed acidity           4898 non-null float64
volatile acidity        4898 non-null float64
citric acid             4898 non-null float64
residual sugar          4898 non-null float64
chlorides               4898 non-null float64
free sulfur dioxide     4898 non-null float64
total sulfur dioxide    4898 non-null float64
density                 4898 non-null float64
pH                      4898 non-null float64
sulphates               4898 non-null float64
alcohol                 4898 non-null float64
quality                 4898 non-null int64
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [27]:
data.columns
y = data['quality']
X = data.drop('quality', axis=1)

Отделите целевой признак, разделите обучающую выборку в отношении 7:3 (30% - под оставленную выборку, пусть random_state=17) и отмасштабируйте данные с помощью StandardScaler.

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [31]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Линейная регрессия
Обучите простую линейную регрессию.

In [32]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Вопрос 1: Каковы среднеквадратичные ошибки линейной регрессии на обучающей и отложенной выборках?

In [33]:
y_train_pred = linreg.predict(X_train_scaled)
y_test_pred = linreg.predict(X_test_scaled)

In [34]:
print(f'Mean squared error (train): {mean_squared_error(y_train, y_train_pred)}')
print(f'Mean squared error (test): {mean_squared_error(y_test, y_test_pred)}')

Mean squared error (train): 0.5648982958116774
Mean squared error (test): 0.562561553622836


Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина (учтите, что большие по модулю отрицательные значения коэффициентов тоже говорят о сильном влиянии). Создайте для этого новый небольшой DataFrame.

In [86]:
coefficients = np.abs(linreg.coef_.ravel())
cols = X_train.columns

linreg_coef = pd.DataFrame(data=coefficients.reshape(11, 1), columns=['coef'], index=cols)

In [89]:
linreg_coef.sort_values(by='coef', ascending=False)

,coef
density,0.400258
residual sugar,0.390505
alcohol,0.255748
volatile acidity,0.191070
pH,0.087720
free sulfur dioxide,0.073669
sulphates,0.056607
fixed acidity,0.055695
total sulfur dioxide,0.021225
chlorides,0.020196
